### Step One : Import packages

In [1]:
import pandas as pd
import sqlite3

### Step Two : Connect with database file

In [2]:
conn = sqlite3.connect('D:\Data\Files\SQL\store.db')

### Step Three : Query each tables

##### This is a customer address table

In [3]:
cus_add = pd.read_sql_query("select * from Customer_Address", conn)
cus_add.dtypes

customer_id    object
address_id      int64
line_1         object
city           object
country        object
zipcode         int64
dtype: object

##### This is a customer order products table

In [4]:
cus_ord_pro = pd.read_sql_query("select * from Customer_Order_Products", conn)
cus_ord_pro.dtypes

order_id      object
product_id    object
qty            int64
comment       object
dtype: object

##### This is a customer orders table

In [5]:
cus_ord = pd.read_sql_query("select * from Customer_Orders",conn)
cus_ord.dtypes

order_id                   object
customer_id                object
customer_payment_method    object
order_status_code          object
order_date                 object
paid_date                  object
der_total_order_price      object
other_order_detail         object
ship_date                  object
dtype: object

##### This is a customers table

In [6]:
cus = pd.read_sql_query("select * from Customers",conn)
cus.dtypes

customer_id    object
first_name     object
last_name      object
cust_phone     object
cust_email     object
dtype: object

##### This is a products table

In [7]:
pro = pd.read_sql_query("select * from Products",conn)
pro.dtypes

product_id           object
product_type_code    object
supplier_code        object
product_name         object
dtype: object

##### This is a products price table

In [8]:
pro_pri = pd.read_sql_query("select * from Products_Price",conn)
pro_pri.dtypes

product_id       object
date_from        object
product_price    object
dtype: object

##### This is a suppliers table

In [9]:
sup = pd.read_sql_query("select * from Suppliers",conn)
sup.dtypes

supplier_code    object
supplier_name    object
dtype: object

### Questions !!!

##### 1) Who are the best customer that highest order of your company?

In [10]:
ans1 = pd.read_sql_query('''SELECT
                            first_name,
                            paid_date,
                            SUM(der_total_order_price) totalSpend
                            FROM (
                            SELECT * FROM Customers A
                            LEFT JOIN Customer_Orders B 
                            on A.customer_id=B.customer_id
                            WHERE paid_date is not NULL
                            )
                            GROUP BY 1
                            ORDER BY 3 DESC
                            LIMIT 1;''', conn)
ans1

,first_name,paid_date,totalSpend
0,Kiley,16-Feb-17,549.0


##### 2) Provide SQL Statement for invoice to Mr.Meaghan?

In [11]:
ans2 = pd.read_sql_query('''SELECT
                                E.customer_id,
                                E.first_name,
                                F.order_id,
                                sum(F.product_price*f.qty) totalPrice
                            FROM Customers E
                            INNER JOIN
                                (
                                SELECT 
                                    *
                                FROM Customer_Orders C
                                LEFT JOIN
                                    (
                                    SELECT 
                                        A.product_price,
                                        A.product_id,
                                        B.order_id,
                                        b.qty
                                    FROM (SELECT *
                                            FROM Products_Price
                                            GROUP by product_id
                                            HAVING max(date_from)) A 
                                    LEFT JOIN Customer_Order_Products B 
                                    on A.product_id=B.product_id
                                    ) D
                                on C.order_id=D.order_id
                                ) F
                            ON E.customer_id=F.customer_id
                            WHERE first_name='Meaghan';''', conn)
ans2

,customer_id,first_name,order_id,totalPrice
0,C00005,Meaghan,O00012,277.0


##### 3) Provide SQL Statement to list the address of customer for support DHL team?

In [12]:
ans3 = pd.read_sql_query('''SELECT
                                J.first_name,
                                I.line_1 || ' ' || I.city || ' ' || I.country || ' ' || I.zipcode as Address,
                                J.product_name,
                                j.supplier_name
                                FROM Customer_Address I
                            INNER JOIN
                            (
                            SELECT
                                H.first_name,
                                G.customer_id,
                                G.product_name,
                                G.supplier_name
                            FROM Customers H
                            INNER JOIN
                            (
                            SELECT
                                F.customer_id,
                                E.product_name,
                                E.supplier_name
                            FROM
                            Customer_Orders F
                            INNER JOIN
                            (
                            SELECT
                                C.order_id,
                                D.product_name,
                                D.supplier_name
                            FROM Customer_Order_Products C
                            INNER JOIN
                            (
                            SELECT 
                                A.product_id,
                                A.product_name,
                                B.supplier_name
                            FROM Products A 
                            INNER JOIN Suppliers B 
                            on A.supplier_code=B.supplier_code
                            ) D
                            on C.product_id=D.product_id
                            ) E
                            on F.order_id=E.order_id
                            WHERE ship_date IS NOT NULL
                            ) G
                            on H.customer_id=G.customer_id
                            ) J
                            on i.customer_id=j.customer_id
                            ;''', conn)
ans3

,first_name,Address,product_name,supplier_name
0,Graciela,98 Cinnecticut Ave Nw Chagrin Falls Geauga 44023,Canon Inkjet,Feltz Printing Service
1,Cammy,56 E Morehead St Laredo Webb 78045,Kingston RAM 16GB DDR4,Printing Dimensions
2,Graciela,98 Cinnecticut Ave Nw Chagrin Falls Geauga 44023,HP Inkjet,Feltz Printing Service
3,Gladys,322 New Horizon Blvd Milwaukee Milwaukee 53207,Canon Inkjet,Feltz Printing Service
4,Cammy,56 E Morehead St Laredo Webb 78045,Canon Inkjet,Feltz Printing Service
5,Cammy,56 E Morehead St Laredo Webb 78045,Kingston RAM 16GB DDR4,Printing Dimensions


##### 4) From Information of Data. Please summary lost opportunity of your company?

In [13]:
ans4 = pd.read_sql_query('''SELECT
                            sum(der_total_order_price) totalLost
                            FROM Customer_Orders
                            WHERE order_status_code 
                            IN ('Canceled','Declined');''', conn)
ans4

,totalLost
0,2289.0


##### 5) Please provide SQL statement to find the best seller of the top five product in every month?

In [14]:
ans5 = pd.read_sql_query('''SELECT 
                                A.product_name,
                                B.qty
                            FROM Products A
                            INNER JOIN Customer_Order_Products B
                            on A.product_id=B.product_id
                            ORDER BY qty DESC
                            LIMIT 5;''', conn)
ans5

,product_name,qty
0,Sandisk RAM 8GB DDR4,4
1,Canon Inkjet,3
2,Canon Inkjet,2
3,Canon Inkjet,2
4,HP Inkjet,2
